### Pipeline optimization 
* Objective:
 * Build a stack of numpy arrays depicting the cost distance away from biogas sources
 
* Requirements - Packages
 * NumPy, Pandas, Scikit-Image

* Requirements - Data:
 * CSV of biogas sources, including coordinates and biogas production potential (MMBtu)
 * Cost surface for building pipelines
 
* Output: 
 * Stacked NumPy array

In [ ]:
#Import packages
import numpy as np
import pandas as pd
from skimage import graph
from osgeo import gdal, osr
import geopandas as gpd
from shapely.geometry import Point
from shapely import wkt

In [ ]:
#Read in cost surface
ds =  gdal.Open('../data/MIT_Surface_Subset/MIT_CostSurface_500m.img')
#Get the raster projection
ds_prj = ds.GetProjection()
#Get raster attributes
llx, x_size, x_angle, lly, y_angle, y_size = ds.GetGeoTransform()
#Extract Band1 as the cost array
arrCost = np.array(ds.GetRasterBand(1).ReadAsArray()) / 100
#Print info
print("Lower left coordinate = ({0:.8f},{1:.8f})".format(llx,lly))
print("Pixel size is {0:.8f}(x), {1:.8f}(y)".format(x_size,y_size))
print("Values range from {} to {}".format(arrCost.min(),arrCost.max()))

In [ ]:
#Set zero cost multiplier to high costs (10)
arrCost[arrCost == 0] = 10
arrCostEven = arrCost / arrCost

In [ ]:
#Read in biogas sources (example: Duplin Co)
dfBG =  pd.read_excel('../data/DuplinCountySwineFarmEconomics.xlsx',
                         sheet_name='Duplin County Swine Farm Master').iloc[:,[11,12,-6,-1]]
dfBG.sort_values(by='Total Potential Methane Yield (scf/h)',ascending=False,inplace=True)

In [ ]:
#Convert to a geopandas dataframe (for easy reprojecting)
geom = [Point(xy) for xy in zip(dfBG.iloc[:,1],dfBG.iloc[:,0])]
gdfBG = gpd.GeoDataFrame(dfBG, geometry = geom)
gdfBG.crs = {'init':'epsg:4326'}
#gdfBG.to_file('../data/DuplinSites_LL.shp')

In [ ]:
#gdfBG.to_file('../data/DuplinSites_LL.shp')

In [ ]:
#Re-project the points to match the raster (takes a moment...)
#gdf2 = gdfBG.to_crs(ds_prj)
gdf2 = gpd.read_file('../data/DuplinSites_Prj.shp')
gdf2.shape

In [ ]:
#Compute cell size (in meters) in miles
x_size_miles = x_size / 1609.344 
y_size_miles = y_size / 1609.344

In [ ]:
#Iterate through all records and create a cost distance raster
cd_arrays = []
for i,r in gdf2.iterrows():
    print(".",end='')
    x = r.geometry.x
    y = r.geometry.y
    z = r[3] 
    
    #Get array index from lat long
    xOffset = int(((x - llx)/x_size))
    yOffset = int(((y - lly)/y_size))

    #Create the minimum cost path (MCP) surface
    lc_graph = graph.MCP_Geometric(arrCost * z / 1609.34, sampling=(x_size,y_size))
    #lc_graph = graph.MCP_Geometric(arrCost)# * z, sampling=(x_size_miles,y_size_miles))
    #Compute cost distances away
    cd_array = lc_graph.find_costs(starts=([(yOffset, xOffset)]))[0]
    #Add the initial cost to all cells
    cd_array = cd_array + z
    #Add to stack of arrays
    cd_arrays.append(cd_array)
        
#Create a stack of all arrays
arrStack =np.stack(cd_arrays)

In [ ]:
#Export lowest cost farm to geotiff
bands,height,width = arrStack.shape
drv = gdal.GetDriverByName("GTiff")
dsOut = drv.Create('../scratch/lowest_cost.tif',width,height,1,gdal.GDT_Float32)

arrFirst = arrStack[0]
arrFirst[arrCost == 10] = -9999.9
dsOut.GetRasterBand(1).WriteArray(arrFirst)
dsOut.GetRasterBand(1).SetNoDataValue(-9999.9)
dsOut.SetGeoTransform (ds.GetGeoTransform())
dsOut.SetProjection(ds_prj)
dsOut.FlushCache()

In [ ]:
#Export min of all rasters to geotiff

#Reduce to minimum costs
arrMin = np.amin(arrStack,axis=0)
print(arrMin.min())

#Create the data source object
bands,height,width = arrStack.shape
drv = gdal.GetDriverByName("GTiff")
dsOut = drv.Create('../scratch/min_cost_surface.tif',width,height,1,gdal.GDT_Float32)

#Set no data values 
arrMin[arrCost == 10] = -9999.9

#Write to the data source object
dsOut.GetRasterBand(1).WriteArray(arrMin)
dsOut.SetGeoTransform (ds.GetGeoTransform())
dsOut.SetProjection(ds_prj)
dsOut.GetRasterBand(1).SetNoDataValue(-9999.9)
dsOut.FlushCache()

In [ ]:
#Stack all the arrays
np.save('../data/DuplinStack500m.npy',arrStack)